### Stress testing for historical regimes 

In [30]:
%pip install ib-insync pandas numpy

Note: you may need to restart the kernel to use updated packages.


#### Test connection: 

In [79]:
from ib_insync import IB, util, Stock, Forex, Future
import pandas as pd
import numpy as np
import os

# This makes ib_insync notebook-friendly (patches nested event loops)
util.startLoop()

In [80]:
### IBKR -> positions + prices + weights 
## handles ES/MES by setting exchange=CME) 

async def fetch_ibkr_positions_with_weights_async(
    host="127.0.0.1",
    port=7497,
    clientId=12,
    currency="USD",
    mkt_data_type=3,            # use delayed prices (15 - 20 mins) when real-time market data (which requires subscription) isnt available
                                # stops failing for stocks/ ETFs
    default_fut_exchange="CME", # for ES (E-mini S&P 500 futures) / MES (Micro E-mini S&P 500 futures) 
                                # specify here so that IBKR can uniquely identify the contract 
                                # avoids error "please enter exchange" 
    weight_mode="gross"         # How portfolio risk is distributed across positions, ignoring cash.
):
    
    ib = IB()
    await ib.connectAsync(host, port, clientId=clientId)

    # Use delayed quotes
    ib.reqMarketDataType(mkt_data_type)

    pos = ib.positions()
    if not pos:
        ib.disconnect()
        raise ValueError("No open positions found in IBKR.")

    # Build a table of positions
    rows = []
    contracts = []
    for p in pos:
        c = p.contract

        # Futures need an exchange (Error 321)
        if getattr(c, "secType", None) == "FUT":
            if not getattr(c, "exchange", None) or c.exchange in ("", "SMART"):
                c.exchange = default_fut_exchange

        # Some contracts may also need currency set; keep existing if already set
        if not getattr(c, "currency", None):
            c.currency = currency

        rows.append({
            "symbol": getattr(c, "symbol", None),
            "secType": getattr(c, "secType", None),
            "exchange": getattr(c, "exchange", None),
            "currency": getattr(c, "currency", None),
            "conId": getattr(c, "conId", None),
            "localSymbol": getattr(c, "localSymbol", None),
            "lastTradeDateOrContractMonth": getattr(c, "lastTradeDateOrContractMonth", None),
            "multiplier": float(getattr(c, "multiplier", 1) or 1),
            "quantity": float(p.position),
        })
        contracts.append(c)

    df = pd.DataFrame(rows)

    # Qualify contracts (fills missing fields and ensures they're requestable)
    qualified = await ib.qualifyContractsAsync(*contracts)

    # Snapshot market data
    tickers = await ib.reqTickersAsync(*qualified)

    def pick_price(t):
        px = t.marketPrice()
        if px is None or (isinstance(px, float) and np.isnan(px)):
            px = t.last
        if px is None or (isinstance(px, float) and np.isnan(px)):
            px = t.close
        return px

    price_map = {t.contract.conId: pick_price(t) for t in tickers}
    df["price"] = df["conId"].map(price_map).astype(float)

    # Signed position value
    df["position_value"] = df["quantity"] * df["price"] * df["multiplier"]

    # Net liquidation value (for net weights)
    nlv = None
    for x in ib.accountSummary():
        if x.tag == "NetLiquidation" and x.currency == currency:
            try:
                nlv = float(x.value)
            except Exception:
                nlv = None
            break

    # Weights
    if weight_mode == "net":
        if not nlv or nlv == 0:
            ib.disconnect()
            raise ValueError("NetLiquidation is missing/0; cannot compute net weights.")
        df["weight"] = df["position_value"] / nlv
    else:  # gross
        gross = df["position_value"].abs().sum()
        if gross == 0 or np.isnan(gross):
            ib.disconnect()
            raise ValueError("Gross exposure is 0/NaN; cannot compute gross weights.")
        df["weight"] = df["position_value"] / gross

    # Helpful diagnostics: which rows still failed to price
    df["priced_ok"] = df["price"].notna()

    # Keep only rows that have a usable price
    df_ok = df[df["priced_ok"]].copy()

    # Recompute weights using only priced positions
    gross = df_ok["position_value"].abs().sum()
    df_ok["weight"] = df_ok["position_value"] / gross

    # show what got dropped
    df_bad = df[~df["priced_ok"]].copy()

    ib.disconnect()  # IMPORTANT: close the connection cleanly

    return df_ok, df_bad

In [81]:
USE_IBKR = True
IBKR_HOST = "127.0.0.1"
IBKR_PORT = 7497
IBKR_CLIENT_ID = 12

if USE_IBKR:
    positions_ok, positions_bad = await fetch_ibkr_positions_with_weights_async(
        host=IBKR_HOST,
        port=IBKR_PORT,
        clientId=IBKR_CLIENT_ID,
        mkt_data_type=3,                 # delayed
        default_fut_exchange="CME",      # fixes ES/MES Error 321
        weight_mode="gross"              # gross weights for stress testing
    )
    # keep only priced positions 
    portfolio = positions_ok[["symbol", "weight"]].copy()
else:
    raise RuntimeError("Set USE_IBKR=True for IBKR-only portfolio.")

# View results (and see any remaining pricing failures)
display(positions_ok[["symbol","secType","exchange","localSymbol","quantity","multiplier","price","position_value","weight","priced_ok"]])
display(positions_bad[["symbol","secType","exchange","localSymbol","quantity","multiplier","price","position_value","priced_ok"]])
display(portfolio)

Error 10089, reqId 14: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.META NASDAQ.NMS/TOP/ALL, contract: Stock(conId=107113386, symbol='META', exchange='NASDAQ', primaryExchange='NASDAQ', currency='USD', localSymbol='META', tradingClass='NMS')
Error 10089, reqId 15: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.TSLA NASDAQ.NMS/TOP/ALL, contract: Stock(conId=76792991, symbol='TSLA', exchange='NASDAQ', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')
Error 10089, reqId 17: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.MSFT NASDAQ.NMS/TOP/ALL, contract: Stock(conId=272093, symbol='MSFT', exchange='NASDAQ', primaryExchange='NASDAQ', currency='USD', localSymbol='MSFT', tradingClass='NMS')
Error 10089, reqId 12: Requested ma

,symbol,secType,exchange,localSymbol,quantity,multiplier,price,position_value,weight,priced_ok
1,G3B,STK,SGX,G3B,2226.0,1.0,5.046,11232.396,0.003067,True
2,META,STK,NASDAQ,META,196.0,1.0,634.980,124456.080,0.033987,True
3,TSLA,STK,NASDAQ,TSLA,-98.0,1.0,412.650,-40439.700,-0.011044,True
4,ES3,STK,SGX,ES3,12588.0,1.0,4.945,62247.660,0.016999,True
5,MSFT,STK,NASDAQ,MSFT,98.0,1.0,399.590,39159.820,0.010694,True
6,IBIT,STK,NASDAQ,IBIT,-234.0,1.0,38.430,-8992.620,-0.002456,True
7,MES,FUT,CME,MESU6,-8.0,5.0,6933.500,-277340.000,-0.075738,True
8,ES,FUT,CME,ESM6,9.0,50.0,6884.375,3097968.750,0.846015,True


,symbol,secType,exchange,localSymbol,quantity,multiplier,price,position_value,priced_ok
0,MSFT,STK,TSE,MSFT,98.0,1.0,NaN,NaN,False


,symbol,weight
1,G3B,0.003067
2,META,0.033987
3,TSLA,-0.011044
4,ES3,0.016999
5,MSFT,0.010694
6,IBIT,-0.002456
7,MES,-0.075738
8,ES,0.846015


The above codes pulls all open positions from IBKR and computes portfolio weights using gross exposure, sourcing prices from IBKR. Delayed market data is enabled to allow pricing without real-time subscriptions, and futures contracts (ES and MES) are explicitly qualified on the CME to ensure correct identification. Positions that cannot be priced due to missing market-data entitlements (e.g. MSFT listed on the Toronto Stock Exchange) are flagged and excluded from the weight calculation.

Final output reports a priced-only portfolio with normalised gross weights that sum to one, reflecting the distribution of risk across the tradable positions.

In [82]:
### Contract builder to construct appropriate IBKR contract object for each asset based on its asset class

def make_contract(row: pd.Series):
    # when pulling from IBKR positions directly, we already have a fully-specified contract.
    # If a contract object exists in the row, use it as-is (preferred).
    if "contract" in row and row["contract"] is not None:
        c = row["contract"]

        # Futures still need an exchange to be requestable (avoids "Please enter exchange")
        if getattr(c, "secType", None) == "FUT":
            if not getattr(c, "exchange", None) or c.exchange in ("", "SMART"):
                c.exchange = "CME"  # for ES / MES

        return c

    # Fallback: if ever pass in a row without a contract object, build one from fields
    symbol = str(row["symbol"]).strip()
    asset_class = str(row.get("asset_class", "")).strip().upper()
    exchange = str(row.get("exchange", "")).strip()
    currency = str(row.get("currency", "USD")).strip()

    if asset_class in {"FX", "FOREX"}:
        # e.g. "EURUSD"
        return Forex(symbol)
    elif asset_class in {"FUT", "FUTURES"}:
        # futures need exchange specified to uniquely identify the contract
        expiry = str(row.get("lastTradeDateOrContractMonth", "")).strip()
        fut_exchange = exchange if exchange else "CME"
        return Future(symbol, lastTradeDateOrContractMonth=expiry, exchange=fut_exchange, currency=currency)
    else:
        # ETFs/stocks (SPY, TLT, GLD)
        # For US ETFs, exchange ARCA is fine.
        return Stock(symbol, exchange, currency)


In [83]:
### Robust daily history puller (chunks by 1Y to avoid IBKR limits) 

### This function retrieves historical daily price data for a given asset from IBKR by splitting the request into one-year chunks to avoid IBKR’s historical data limits. 
### Starting from the specified end date, it repeatedly requests one year of daily bars, converts each response into a DataFrame, and moves backwards in time until the start date is reached or no more data is available. 
### The individual chunks are then combined, de-duplicated, sorted chronologically, and trimmed to the exact date range requested. 

async def fetch_daily_history(
    ib: IB,
    contract,
    start_date: str,
    end_date: str,
    whatToShow: str,
    useRTH: bool,
    chunk: str = "1 Y",
    barSizeSetting: str = "1 day",
):
    """
    Pull daily bars between [start_date, end_date] inclusive.
    Uses chunking (1 year per request) and stitches results together.

    since we now pull positions/contracts directly from IBKR,
    we expect 'contract' to already be an IBKR contract object. We still qualify it to
    ensure it is requestable (and to avoid common validation issues such as missing exchange
    for futures).
    """
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)

    # ensure contract is fully specified / requestable before pulling history
    # (helps avoid errors like "Please enter exchange" for futures)
    qualified = await ib.qualifyContractsAsync(contract)
    if not qualified:
        return pd.DataFrame()
    contract = qualified[0]

    all_chunks = []
    end_dt = end

    # Loop backwards from end_date in chunks until we cover start_date
    while end_dt >= start:
        bars = await ib.reqHistoricalDataAsync(
            contract,
            endDateTime=end_dt.strftime("%Y%m%d 23:59:59"),
            durationStr=chunk,
            barSizeSetting=barSizeSetting,
            whatToShow=whatToShow,
            useRTH=useRTH,
            formatDate=1,
        )

        if not bars:
            break

        df = util.df(bars)
        
        if df is None or df.empty:
            break

        df["date"] = pd.to_datetime(df["date"])
        all_chunks.append(df)

        # Move the end pointer to just before the earliest bar we received
        earliest = df["date"].min()
        end_dt = earliest - pd.Timedelta(days=1)

    if not all_chunks:
        return pd.DataFrame()

    out = pd.concat(all_chunks, ignore_index=True)
    out = out.drop_duplicates(subset=["date"]).sort_values("date")
    out = out[(out["date"] >= start) & (out["date"] <= end)].reset_index(drop=True)
    return out


In [84]:
### qualify + pull + save for each asset 

### This function converts each portfolio entry into an IBKR contract, qualifies those contracts to ensure they are uniquely identified and usable by the API, 
### and then iterates through each asset to pull its daily historical data over the specified date range. Depending on the asset class, it requests the appropriate data type 
### (midpoint prices for FX and trade prices for equities/ETFs) and uses the robust chunked history function to handle IBKR data limits. 
### The resulting price series for each asset is stored in a dictionary and saved as a CSV file. 

async def pull_all_assets_history(
    ib: IB,
    portfolio_df: pd.DataFrame,
    start_date: str,
    end_date: str,
    out_dir: str = "outputs_prices",
):
    os.makedirs(out_dir, exist_ok=True)

    results = {}

    # portfolio entries now come from IBKR positions (positions_ok),
    # so we infer what contract to build using secType (STK/FUT/CASH).
    def build_contract_from_ibkr_row(row: pd.Series):
        sym = str(row.get("symbol", "")).strip()
        secType = str(row.get("secType", "")).strip().upper()
        exchange = str(row.get("exchange", "")).strip()
        currency = str(row.get("currency", "USD")).strip()
        expiry = str(row.get("lastTradeDateOrContractMonth", "")).strip()

        if secType in {"CASH", "FX", "FOREX"}:
            # IBKR FX positions typically show symbol='EUR', currency='USD' etc.
            # Convert to pair string like "EURUSD"
            return Forex(f"{sym}{currency}")
        elif secType == "FUT":
            # Futures MUST be Future(...), not Stock(...), and need expiry + exchange
            fut_exchange = exchange if exchange and exchange not in ("SMART",) else "CME"
            if not expiry:
                # If expiry is missing, IBKR can't uniquely define the futures contract
                return None
            return Future(sym, lastTradeDateOrContractMonth=expiry, exchange=fut_exchange, currency=currency)
        else:
            # Stocks/ETFs
            # For US ETFs/stocks, SMART is usually fine; exchange can be kept if already specified (NASDAQ/SGX/etc.)
            stk_exchange = exchange if exchange else "SMART"
            return Stock(sym, stk_exchange, currency)

    # Iterate through each asset and pull history (qualify per-asset to avoid index mismatch)
    for _, row in portfolio_df.iterrows():
        sym = str(row.get("symbol", "")).strip()

        contract = build_contract_from_ibkr_row(row)
        if contract is None:
            print(f"Skipped {sym}: missing fields to build a valid contract (e.g., futures expiry).")
            continue

        # Qualify contract (fills conId etc.)
        qualified = await ib.qualifyContractsAsync(contract)
        if not qualified:
            print(f"Skipped {sym}: contract could not be qualified -> {contract}")
            continue
        contract = qualified[0]

        # Decide whatToShow/useRTH based on asset class / secType
        secType = str(getattr(contract, "secType", "")).strip().upper()
        if secType in {"CASH"}:
            whatToShow = "MIDPOINT"
            useRTH = False
        else:
            whatToShow = "TRADES"
            useRTH = True

        df = await fetch_daily_history(
            ib=ib,
            contract=contract,
            start_date=start_date,
            end_date=end_date,
            whatToShow=whatToShow,
            useRTH=useRTH,
        )

        results[sym] = df

        # Save
        out_path = os.path.join(out_dir, f"{sym}_{start_date}_{end_date}.csv".replace(":", "-"))
        df.to_csv(out_path, index=False)
        print(f"Saved {sym}: {len(df)} rows -> {out_path}")

    return results


In [85]:
### Run 

START_DATE = "2007-01-01"
END_DATE   = (pd.Timestamp.today().normalize() - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
print("Using END_DATE =", END_DATE)

# IMPORTANT:
# We now pass the priced IBKR positions (positions_ok),
# not a static portfolio.csv.
# This ensures historical prices are pulled only for valid, tradable positions.

from ib_insync import IB

ib = IB()
await ib.connectAsync(IBKR_HOST, IBKR_PORT, clientId=IBKR_CLIENT_ID)

prices = await pull_all_assets_history(
    ib=ib,
    portfolio_df=positions_ok,   
    start_date=START_DATE,
    end_date=END_DATE,
)

Using END_DATE = 2026-02-16


Error 162, reqId 20: Historical Market Data Service error message:HMDS query returned no data: G3B@SGX Trades, contract: Stock(conId=92213911, symbol='G3B', exchange='SGX', primaryExchange='SGX', currency='SGD', localSymbol='G3B', tradingClass='G3B')


Saved G3B: 3627 rows -> outputs_prices/G3B_2007-01-01_2026-02-16.csv


Error 162, reqId 37: Historical Market Data Service error message:HMDS query returned no data: META@NASDAQ Trades, contract: Stock(conId=107113386, symbol='META', exchange='NASDAQ', primaryExchange='NASDAQ', currency='USD', localSymbol='META', tradingClass='NMS')


Saved META: 3455 rows -> outputs_prices/META_2007-01-01_2026-02-16.csv


...

Saved IBIT: 525 rows -> outputs_prices/IBIT_2007-01-01_2026-02-16.csv


Error 162, reqId 106: Historical Market Data Service error message:HMDS query returned no data: MESU6@CME Trades, contract: Future(conId=793356217, symbol='MES', lastTradeDateOrContractMonth='20260918', multiplier='5', exchange='CME', currency='USD', localSymbol='MESU6', tradingClass='MES')


Saved MES: 164 rows -> outputs_prices/MES_2007-01-01_2026-02-16.csv


Error 162, reqId 112: Historical Market Data Service error message:HMDS query returned no data: ESM6@CME Trades, contract: Future(conId=649180678, symbol='ES', lastTradeDateOrContractMonth='20260618', multiplier='50', exchange='CME', currency='USD', localSymbol='ESM6', tradingClass='ES')


Saved ES: 618 rows -> outputs_prices/ES_2007-01-01_2026-02-16.csv


(1) IBKR returns Error 162 (“HMDS query returned no data”) during historical price pulls

This occurs when a chunked historical-data request reaches a period for which IBKR has no available bars. Common reasons include the request extending beyond an instrument’s listing date (e.g. META, TSLA, IBIT, or current ES/MES futures contracts), exchange-specific coverage limits (e.g. SGX equities), or the final backward chunk falling outside IBKR’s historical database. In these cases, IBKR correctly signals that no further data can be retrieved for that asset.

(2) The data-pull logic handles this condition and continues execution

The chunked history function treats “no data” responses as a stopping condition. It exits the backward-fetch loop, preserves all valid historical chunks collected up to that point, and writes the resulting time series to CSV.

In [86]:
### for checking purpose 

prices["G3B"].tail()

,date,open,high,low,close,volume,average,barCount
3622,2026-02-10,5.048,5.070,5.039,5.050,371495.0,5.0511,202
3623,2026-02-11,5.054,5.080,5.040,5.069,209839.0,5.0668,239
3624,2026-02-12,5.080,5.120,5.080,5.115,371479.0,5.1092,181
3625,2026-02-13,5.113,5.113,5.035,5.039,532814.0,5.0555,505
3626,2026-02-16,5.073,5.096,5.020,5.046,2386470.0,5.0367,214


In [87]:
### paths 

SCENARIOS_PATH = "scenarios.csv"
PRICES_DIR = "outputs_prices"

### load inputs
scenarios = pd.read_csv(SCENARIOS_PATH)
scenarios.columns = [c.strip() for c in scenarios.columns]

scenarios

,scenario,start_date,end_date
0,afc,1997-07-01,1998-12-31
1,russian_default_ltcm,1998-08-01,1998-10-31
2,dot_com,2000-03-01,2002-10-31
3,gfc,2007-08-01,2009-06-30
4,flash_crash,2010-05-06,2010-05-07
5,fukushima_meltdown,2011-03-01,2011-04-30
6,sp_downgrade,2011-08-01,2011-08-31
7,euro_debt_crisis,2014-08-01,2014-12-31
8,taper_tantrum,2013-05-01,2013-09-30
9,a50_turbulence,2015-06-01,2016-02-29


In [88]:
### load all price data 

price_data = {}

for sym in positions_ok["symbol"].unique():
    files = [f for f in os.listdir(PRICES_DIR) if f.startswith(sym + "_")]

    # Skip symbols with no saved price file (defensive check)
    if not files:
        print(f"No price file found for {sym}, skipping.")
        continue

    file = files[0]
    df = pd.read_csv(os.path.join(PRICES_DIR, file))
    df["date"] = pd.to_datetime(df["date"])

    price_data[sym] = df.set_index("date").sort_index()

price_data.keys()


dict_keys(['G3B', 'META', 'TSLA', 'ES3', 'MSFT', 'IBIT', 'MES', 'ES'])

In [89]:
### helper function to compute stress return (for one asset only) 
### it handles weekends / holidays and uses nearest trading day ≤ start/end (using the last available price before the date avoids look-ahead bias)

def stress_return_from_prices(px: pd.DataFrame, start_date: str, end_date: str):
    s = pd.Timestamp(start_date)
    e = pd.Timestamp(end_date)

    # IBKR historical bars (via util.df) typically use lowercase 'close'
    # Keep this robust in case CSV columns differ.
    close_col = "close" if "close" in px.columns else ("Close" if "Close" in px.columns else None)
    if close_col is None:
        return None  # missing close prices

    # nearest trading days ≤ start/end
    try:
        start_px = px.loc[:s].iloc[-1][close_col]
        end_px   = px.loc[:e].iloc[-1][close_col]
    except IndexError:
        return None  # not enough data

    return end_px / start_px - 1


In [94]:
# create a clean portfolio table from IBKR-priced positions
# (collapses duplicates like MSFT across exchanges if they exist)

portfolio = (
    positions_ok.groupby("symbol", as_index=False)["weight"]
    .sum()
)

### Compute asset-level stress returns
### produces one row per (scenario × asset

### Asset return = percentage price change over scenario window (independent of how large the position is in the portfolio)
### ie. (Price at end of scenario / Price at start of scenario) − 1

### Contribution = how many percentage points this asset added to or subtracted from the portfolio
### ie. weight × asset_return

asset_results = []

for _, sc in scenarios.iterrows():
    scenario = sc["scenario"]
    start = sc["start_date"]
    end = sc["end_date"]

    for _, row in portfolio.iterrows():
        sym = row["symbol"]
        weight = row["weight"]

        # New: defensive check in case a price series failed to save/load
        if sym not in price_data:
            r = None
        else:
            px = price_data[sym]
            r = stress_return_from_prices(px, start, end)

        asset_results.append({
            "scenario": scenario,
            "symbol": sym,
            "weight": weight,
            "asset_return": r,
            "contribution": None if r is None else weight * r
        })

asset_results = pd.DataFrame(asset_results)
asset_results[:50]


,scenario,symbol,weight,asset_return,contribution
0,afc,ES,0.846015,NaN,NaN
1,afc,ES3,0.016999,NaN,NaN
2,afc,G3B,0.003067,NaN,NaN
3,afc,IBIT,-0.002456,NaN,NaN
4,afc,MES,-0.075738,NaN,NaN
5,afc,META,0.033987,NaN,NaN
6,afc,MSFT,0.010694,NaN,NaN
7,afc,TSLA,-0.011044,NaN,NaN
8,russian_default_ltcm,ES,0.846015,NaN,NaN
9,russian_default_ltcm,ES3,0.016999,NaN,NaN


The NaNs mostly happen because the asset doesn’t have price history covering the full scenario window (e.g., the stock/ETF didn’t exist yet, or the start date is before listing)

In [95]:
import yfinance as yf

### load all price data (from yfinance instead of IBKR CSVs)

# Map IBKR symbols -> Yahoo tickers (edit these as needed)
# US stocks/ETFs are usually the same symbol (MSFT, TSLA, META, IBIT).
# SGX tickers often need ".SI" on Yahoo (not always guaranteed).
# Futures on Yahoo use special tickers (ES=F). Micro futures may not exist on Yahoo.

YF_TICKER_MAP = {
    "ES": "ES=F",      # E-mini S&P 500 futures (Yahoo)
    "MES": "MES=F",    # may or may not exist on Yahoo; if it fails, get NaNs
    "G3B": "G3B.SI",   # SGX (common pattern)
    "ES3": "ES3.SI",   # SGX (common pattern)
    # "MSFT": "MSFT",
    # "META": "META",
    # "TSLA": "TSLA",
    # "IBIT": "IBIT",
}

def yf_download_daily(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Downloads daily OHLCV from yfinance and returns a DataFrame with a DatetimeIndex
    and a lowercase 'close' column (to match stress_return_from_prices).
    """
    df = yf.download(
        ticker,
        start=start_date,
        end=pd.Timestamp(end_date) + pd.Timedelta(days=1),  # make end inclusive-ish
        interval="1d",
        auto_adjust=False,
        progress=False,
        threads=True,
        group_by="column",   # helps reduce MultiIndex surprises
    )

    if df is None or df.empty:
        return pd.DataFrame()

    # If columns are MultiIndex (tuples), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]  # keep just "Open/High/Low/Close/Adj Close/Volume"

    df = df.reset_index()

    # Standardize Date column name
    if "Date" in df.columns:
        df.rename(columns={"Date": "date"}, inplace=True)
    elif "Datetime" in df.columns:
        df.rename(columns={"Datetime": "date"}, inplace=True)

    df["date"] = pd.to_datetime(df["date"])

    # Now safe to lowercase
    df.columns = [str(c).lower() for c in df.columns]

    return df.set_index("date").sort_index()

price_data = {}

# New: use symbols from IBKR-priced positions (or your `portfolio` table if that's what you use)
symbols = positions_ok["symbol"].unique()

for sym in symbols:
    yf_ticker = YF_TICKER_MAP.get(sym, sym)  # default: same as symbol
    df = yf_download_daily(yf_ticker, START_DATE, END_DATE)

    if df.empty or ("close" not in df.columns):
        print(f"[yfinance] No usable data for {sym} (mapped to {yf_ticker}) -> will produce NaNs")
        continue

    price_data[sym] = df

price_data.keys()


dict_keys(['G3B', 'META', 'TSLA', 'ES3', 'MSFT', 'IBIT', 'MES', 'ES'])

In [96]:
### Drop rows where asset-level stress returns could not be computed
### (e.g. insufficient price history, asset did not exist during scenario window,
### or price data unavailable from IBKR / yfinance)

asset_results_updated = (
    asset_results
    .dropna(subset=["asset_return", "contribution"])
    .reset_index(drop=True)
)

### Omit scenarios with insufficient asset coverage 

coverage = (
    asset_results_updated.groupby("scenario")["symbol"]
    .nunique()
    .reset_index(name="n_assets_with_data")
)

# drop scenarios with <= 1 asset available
min_assets_required = 2

bad_scenarios = coverage.loc[coverage["n_assets_with_data"] < min_assets_required, "scenario"].tolist()
print("Dropping scenarios (insufficient data):", bad_scenarios)

asset_results_updated = asset_results_updated[~asset_results_updated["scenario"].isin(bad_scenarios)].copy()

# show remaining coverage
print("\nRemaining scenario coverage:")
print(
    asset_results_updated.groupby("scenario")["symbol"].nunique().sort_values()
)

Dropping scenarios (insufficient data): []

Remaining scenario coverage:
scenario
gfc                         2
flash_crash                 4
fukushima_meltdown          5
sp_downgrade                5
a50_turbulence              6
brexit                      6
euro_debt_crisis            6
taper_tantrum               6
us_presidential_election    6
volmageddon                 6
covid19                     7
fitch_downgrade             7
global_inflation            7
carry_trade_unwind          8
gold_silver_bust            8
trump_tariffs               8
Name: symbol, dtype: int64


In [97]:
asset_results_updated [:50] 

,scenario,symbol,weight,asset_return,contribution
0,gfc,ES,0.846015,-0.377211,-0.319126
1,gfc,MSFT,0.010694,-0.188737,-0.002018
2,flash_crash,ES,0.846015,-0.013808,-0.011682
3,flash_crash,ES3,0.016999,-0.006897,-0.000117
4,flash_crash,G3B,0.003067,0.000000,0.000000
5,flash_crash,MSFT,0.010694,-0.026570,-0.000284
6,fukushima_meltdown,ES,0.846015,0.045158,0.038204
7,fukushima_meltdown,ES3,0.016999,0.035256,0.000599
8,fukushima_meltdown,G3B,0.003067,0.042071,0.000129
9,fukushima_meltdown,MSFT,0.010694,-0.009174,-0.000098


In [99]:
import pandas as pd

# 1) Shock matrix
shock = (asset_results_updated
         .pivot_table(index="scenario", columns="symbol", values="asset_return", aggfunc="first"))

# 2) Scenario metadata (read from existing scenarios.csv)

scenarios_out = scenarios.copy()
scenarios_out["start_date"] = pd.to_datetime(scenarios_out["start_date"])
scenarios_out["end_date"] = pd.to_datetime(scenarios_out["end_date"])

# Save
shock.to_parquet("data/processed/shock_matrix.parquet")
scenarios_out.to_parquet("data/processed/scenarios.parquet", index=False)

print("Saved: data/processed/shock_matrix.parquet")
print("Saved: data/processed/scenarios.parquet")


Saved: data/processed/shock_matrix.parquet
Saved: data/processed/scenarios.parquet


Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Peer closed connection.


In [71]:
### compute portfolio-level stress returns, which is the sum of all asset contributions in a scenario 
### this summarize how the entire portfolio would have performed during a historical crisis, rather than how individual assets moved in isolation. 

portfolio_stress_results = (
    asset_results_updated
    .groupby("scenario", as_index=False)
    .agg(
        portfolio_return=("contribution", "sum")
    )
)

portfolio_stress_results

,scenario,portfolio_return
0,a50_turbulence,-0.061821
1,brexit,-0.003978
2,carry_trade_unwind,0.027335
3,covid19,0.178280
4,euro_debt_crisis,0.063294
5,fitch_downgrade,0.005620
6,flash_crash,-0.012085
7,fukushima_meltdown,0.037144
8,gfc,-0.321173
9,global_inflation,0.017084


In [72]:
### rank scenarios by severity 

portfolio_stress_results = portfolio_stress_results.sort_values(
    by="portfolio_return"
)

portfolio_stress_results

,scenario,portfolio_return
8,gfc,-0.321173
0,a50_turbulence,-0.061821
11,sp_downgrade,-0.041620
15,volmageddon,-0.034947
6,flash_crash,-0.012085
1,brexit,-0.003978
13,trump_tariffs,0.003968
5,fitch_downgrade,0.005620
10,gold_silver_bust,0.011589
9,global_inflation,0.017084


COVID-19 emerges as the most severe stress event, reflecting the portfolio’s heavy exposure to equity beta, particularly via S&P 500 futures (ES3) and high-growth equities. Other large macro shocks such as the dot-com crash and Trump-era tariff also produce negative outcomes, but to a substantially smaller degree, suggesting that the portfolio is more vulnerable to sudden global liquidity and risk-aversion shocks than to slow-burn structural crises. 

Conversely, several historically disruptive events—including the euro debt crisis, global inflation, and China-specific turbulence—are associated with positive portfolio outcomes. Overall, the ranking highlights a portfolio that performs well in most environments but remains highly sensitive to sharp equity drawdowns. 

In [73]:
### identify main loss drivers per scenario 

top_contributors = (
    asset_results_updated
    .sort_values(["scenario", "contribution"])
    .groupby("scenario")
    .head(1)   # top worst contributor per scenario
)

top_contributors

,scenario,symbol,weight,asset_return,contribution
31,a50_turbulence,ES,0.846089,-0.085220,-0.072104
37,brexit,ES,0.846089,-0.003694,-0.003125
80,carry_trade_unwind,MES,-0.075746,0.032982,-0.002498
61,covid19,TSLA,-0.011073,4.282040,-0.047415
21,euro_debt_crisis,G3B,0.003066,0.011730,0.000036
72,fitch_downgrade,MES,-0.075746,0.006183,-0.000468
5,flash_crash,ES,0.846089,-0.013808,-0.011683
13,fukushima_meltdown,TSLA,-0.011073,0.152882,-0.001693
3,gfc,ES,0.846089,-0.377211,-0.319154
65,global_inflation,MES,-0.075746,0.012924,-0.000979


The table reinforces that severity at the portfolio level is explained by a small number of large, concentrated contributors, rather than uniform losses across assets. 

In severe scenarios, losses are dominated by either S&P 500 futures (ES) or high-volatility equities such as TSLA, whose large price swings translate into outsized contributions once weighted. For example, during COVID-19 and taper tantrum episodes, TSLA exhibits extremely large positive or negative asset returns, but because the position is held short, adverse moves translate into significant negative portfolio contributions. 

Similarly, in trade-related shocks such as Trump tariffs, the dominant drag comes from the ES position, reflecting the portfolio’s high gross exposure to equity index futures. In contrast, smaller positions such as G3B or MSFT tend to contribute marginally to portfolio outcomes even when their asset-level returns are large, highlighting that weight, not volatility alone, determines stress impact. This decomposition shows that portfolio stress is concentrated in a few risk factors—primarily equity beta and high-growth equity exposure—rather than being broadly diversified across assets.

Own remarks: 
 
position_value = current dollar exposure of a single position
- position_value = quantity * last_price * multiplier 
- multiplier = contract-specific scaling factor

Net weights (sum can be < 1 if cash exists)
* weight_net_i = position_value_i / net_liquidation_value
- i = index for each asset in portfolio 
- weight_net_i = fraction of your total portfolio capital allocated to asset i
- net_liquidation_value = amount of money you would have if you closed all positions immediately at current market prices.

Gross exposure weights (separates long & short properly)
* gross = sum(abs(position_value))
* weight_gross_i = position_value_i / gross
* Long book weight: 
* w_long_i = position_value_i / sum(long_values)
* Short book weight: 
* w_short_i = abs(position_value_i) / sum(abs(short_values))


Simple glossary: 

1. afc = The Asian Financial Crisis (1997–1998), triggered by currency devaluations and capital flight across East and Southeast Asia, leading to sharp equity market declines and regional financial instability.
2. russian_default_ltcm = Russia’s 1998 sovereign debt default combined with the collapse of Long-Term Capital Management, which caused severe stress in global bond markets and highlighted systemic leverage risk.
3. dot_com = The bursting of the late-1990s technology bubble, marked by a prolonged collapse in equity valuations following excessive speculation in internet and growth stocks.
4. gfc = The Global Financial Crisis (2007–2009), driven by the US subprime mortgage collapse and failures in the global banking system, resulting in a deep and broad-based market downturn.
5. flash_crash = The May 2010 Flash Crash, a sudden and extreme intraday equity market sell-off caused by liquidity evaporation and automated trading feedback loops.
6. fukushima_meltdown = The 2011 Fukushima nuclear disaster following a major earthquake and tsunami in Japan, which disrupted global supply chains and triggered risk-off market behaviour.
7. sp_downgrade = The 2011 downgrade of US sovereign credit by Standard & Poor’s, which increased market uncertainty despite US Treasuries retaining safe-haven status.
8. euro_debt_crisis = The European sovereign debt crisis, characterised by concerns over fiscal sustainability in countries such as Greece, Spain, and Italy, leading to stress in European assets and currencies.
9. taper_tantrum = The 2013 market sell-off following the US Federal Reserve’s signal of tapering quantitative easing, which caused a sharp rise in bond yields and losses in duration-sensitive assets.
10. a50_turbulence = The 2015–2016 turbulence in Chinese equity markets, including sharp declines in the China A-share market and heightened volatility linked to growth concerns and policy uncertainty.
11. brexit = The 2016 UK referendum decision to leave the European Union, which led to immediate currency depreciation and heightened volatility across European financial markets.
12. us_presidential_election = The 2016 US presidential election, which generated short-term market volatility driven by policy uncertainty and shifts in investor expectations.
13. volmageddon = The February 2018 volatility spike, during which inverse volatility products collapsed and equity markets experienced abrupt losses.
14. covid19 = The global COVID-19 pandemic, which caused an abrupt economic shutdown and extreme market volatility, followed by strong policy-driven asset price recoveries.
15. global_inflation = The post-pandemic global inflation surge, characterised by rising interest rates and tightening monetary policy, negatively impacting bonds and growth-oriented assets.
16. fitch_downgrade = Fitch’s 2023 downgrade of US sovereign credit, reflecting concerns over fiscal governance and contributing to short-term market volatility.
17. carry_trade_unwind = A rapid unwinding of global carry trades, typically driven by rising volatility or funding currency appreciation, leading to losses in leveraged FX positions.
18. trump_tariffs = The escalation of US–China trade tensions through tariff announcements under the Trump administration, increasing uncertainty for global trade and risk assets.
19. gold_silver_bust = A sharp correction in precious metals prices following a speculative run-up, reflecting shifts in inflation expectations, real yields, or investor positioning.